In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
import json
import torch.nn as nn
import torch.nn.init as init

/home/csgrad/byalavar/miniconda3/envs/adaface/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import matplotlib.pyplot as plt
import torch.nn.functional as F


def twoSum(nums, target):
    d = {}
    for i, num in enumerate(nums):
        if target - num in d:
            return [d[target - num], i]
        d[num] = i
    return []

In [29]:
import numpy as np
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt

def gaussian_kernel_3d(sigma, noise_factor=0.05):
    # Define the size of the kernel
    size = 5
    
    # Generate the kernel using scipy's gaussian_filter function
    kernel = np.zeros((size, size, size))
    kernel[size // 2, size // 2, size // 2] = 1
    kernel = gaussian_filter(kernel, sigma)

    # Normalize the kernel to ensure that the sum of all elements is equal to 1
    kernel /= np.sum(kernel)
    
    # Add random noise to the kernel
    noise = np.random.normal(0, noise_factor, kernel.shape)
    kernel += noise
    
    return kernel

# Example usage:
sigma = 2
kernel = gaussian_kernel_3d(sigma)
print("3D Gaussian Kernel with sigma =", sigma, ":\n", kernel)

3D Gaussian Kernel with sigma = 2 :
 [[[-0.01054636 -0.03082467  0.08667448  0.01594209  0.12088814]
  [ 0.05516346  0.03112081 -0.05396956  0.08668332 -0.07461851]
  [ 0.02034032 -0.05556425  0.00913102  0.02867785  0.00123339]
  [-0.02809119  0.03250382 -0.0046246   0.05785382  0.02962006]
  [ 0.01976044 -0.06770883  0.06298323  0.08050761  0.06797029]]

 [[ 0.03249925  0.01382108 -0.06459239 -0.02220664  0.05040374]
  [ 0.06416198 -0.00886387  0.02238148 -0.07698237  0.09136609]
  [ 0.03247358  0.08854344  0.0634775   0.04371369  0.06068973]
  [ 0.01079561 -0.01865739  0.02882798  0.01198771 -0.07749449]
  [ 0.06247396 -0.11812332  0.05925183 -0.0152915   0.02955602]]

 [[ 0.01348276  0.03341142 -0.00846862 -0.00019438  0.04399838]
  [-0.03682561  0.0716253  -0.01793406  0.03009526 -0.00813489]
  [-0.02175859  0.00418994  0.01297543  0.08875282 -0.01126669]
  [-0.05223222 -0.06165323 -0.05812875  0.06653905  0.04476722]
  [ 0.02629568 -0.01998463 -0.01392796  0.02202469  0.07027919]

In [2]:
'''
Properly implemented ResNet-s for CIFAR10 as described in paper [1].

The implementation and structure of this file is hugely influenced by [2]
which is implemented for ImageNet and doesn't have option A for identity.
Moreover, most of the implementations on the web is copy-paste from
torchvision's resnet and has wrong number of params.

Proper ResNet-s for CIFAR10 (for fair comparision and etc.) has following
number of layers and parameters:

name      | layers | params
ResNet20  |    20  | 0.27M
ResNet32  |    32  | 0.46M
ResNet44  |    44  | 0.66M
ResNet56  |    56  | 0.85M
ResNet110 |   110  |  1.7M
ResNet1202|  1202  | 19.4m

which this implementation indeed has.

Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
[2] https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

If you use this implementation in you work, please don't forget to mention the
author, Yerlan Idelbayev.
'''
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

from torch.autograd import Variable

__all__ = ['ResNet', 'resnet20', 'resnet32', 'resnet44', 'resnet56', 'resnet110', 'resnet1202']

def _weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A':
                """
                For CIFAR10 ResNet paper uses option A.
                """
                self.shortcut = LambdaLayer(lambda x:
                                            F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64, num_classes)

        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def resnet20():
    return ResNet(BasicBlock, [3, 3, 3])


def resnet32():
    return ResNet(BasicBlock, [5, 5, 5])


def resnet44():
    return ResNet(BasicBlock, [7, 7, 7])


def resnet56():
    return ResNet(BasicBlock, [9, 9, 9])


def resnet110():
    return ResNet(BasicBlock, [18, 18, 18])


def resnet1202():
    return ResNet(BasicBlock, [200, 200, 200])


def test(net):
    import numpy as np
    total_params = 0

    for x in filter(lambda p: p.requires_grad, net.parameters()):
        total_params += np.prod(x.data.numpy().shape)
    print("Total number of params", total_params)
    print("Total layers", len(list(filter(lambda p: p.requires_grad and len(p.data.size())>1, net.parameters()))))


if __name__ == "__main__":
    for net_name in __all__:
        if net_name.startswith('resnet'):
            print(net_name)
            #test(globals()[net_name]())
            print()

resnet20

resnet32

resnet44

resnet56

resnet110

resnet1202



In [2]:
# Helper Box Filters
import numpy as np
import copy
import matplotlib.pyplot as plt

class boxFilters:
    def __init__(self,ogFilterSize):
        self.alpha = None
        self.a=None
        self.b=None
        self.c=None
        self.d=None
        self.ogFilterSize = ogFilterSize
    
    def setAlpha(self,alpha):
        self.alpha = alpha
    
    def setABCD(self,a,b,c,d):
        self.a = a
        self.b = b
        self.c = c
        self.d = d
    
    def printParams(self):
        print("A:",self.a,"B:",self.b,"C:",self.c,"D:",self.d, "Alpha:", self.alpha)


def overlapping_area(box1, box2):
    # Extract coordinates of the top-left and bottom-right corners of each box
    x1_tl, y1_tl, x1_br, y1_br = box1.a, box1.c, box1.b, box1.d
    x2_tl, y2_tl, x2_br, y2_br = box2.a, box2.c, box2.b, box2.d
    #print(x1_tl, y1_tl, x1_br, y1_br)
    #print(x2_tl, y2_tl, x2_br, y2_br)
    # Calculate coordinates of the intersection rectangle
    x_intersection_tl = max(x1_tl, x2_tl)
    y_intersection_tl = max(y1_tl, y2_tl)
    x_intersection_br = min(x1_br, x2_br)
    y_intersection_br = min(y1_br, y2_br)
    

    # Check if there is an actual intersection
    if x_intersection_tl <= x_intersection_br and y_intersection_tl <= y_intersection_br:
        # Calculate the width and height of the intersection rectangle
        intersection_width = x_intersection_br - x_intersection_tl + 1
        intersection_height = y_intersection_br - y_intersection_tl + 1
        # Calculate the area of the intersection rectangle
       # print(intersection_width,intersection_height)
        intersection_area = intersection_width * intersection_height
        return intersection_area
    else:
        # No intersection
        return 0

def sum_within_boundary(filter, x1, y1, x2, y2):
    # Ensure x1 <= x2 and y1 <= y2
    x1, x2 = min(x1, x2), max(x1, x2)
    y1, y2 = min(y1, y2), max(y1, y2)

    # Ensure the boundary coordinates are within the matrix dimensions
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(len(filter) - 1, x2), min(len(filter[0]) - 1, y2)
   # print("x1:",x1,"y1:",y1,"x2:",x2,"y2:",y2)
    # Calculate the sum within the boundary
    boundary_sum = 0
    for i in range(y1, y2 + 1):
        for j in range(x1, x2 + 1):
            #print(i,j)
            boundary_sum += filter[i][j]
           # print(boundary_sum)
    return boundary_sum



def calcAlpha(boxFiltersList,filter):

    A = np.ones((len(boxFiltersList),len(boxFiltersList)))
    for i in range(0,len(boxFiltersList)):
        for j in range(0,len(boxFiltersList)):
           A[j][i]  =  overlapping_area(boxFiltersList[i],boxFiltersList[j])    
    
    det = np.linalg.det(A)
    #print("A:",A)
    AInv = None
    if np.isclose(det, 0):
        #print("Singular")
        pass
        
    else:
        # Calculate the inverse
        AInv = np.linalg.inv(A)
    
    B = np.ones((len(boxFiltersList)))

    for i in range(0,len(boxFiltersList)):
        B[i] = sum_within_boundary(filter,boxFiltersList[i].a,boxFiltersList[i].c,boxFiltersList[i].b,boxFiltersList[i].d)
    #print("B:",B)
    if(np.all(AInv==None)):
        return None
    else:
     return np.matmul(AInv,B)    
    

def l2_distance(arr1, arr2):
    # Flatten the arrays to treat them as vectors
    vec1 = arr1.flatten()
    vec2 = arr2.flatten()
    
    # Calculate the L2 distance
    distance = np.linalg.norm(vec1 - vec2)
    
    return distance

def percentage_l2_error(matrix1, matrix2):
    # Calculate the L2 norm of the difference between the matrices
    diff_norm = np.linalg.norm(matrix1 - matrix2)
    #print(diff_norm)
    # Calculate the L2 norm of one of the matrices
    norm_matrix1 = np.linalg.norm(matrix1)

    # Calculate the percentage L2 error
    percent_error = (diff_norm / norm_matrix1) * 100

    return percent_error



def computeTempFilter(tempBoxFilterList):
    tempFilter = np.zeros((tempBoxFilterList[0].ogFilterSize,tempBoxFilterList[0].ogFilterSize))
    for i in range(0,len(tempBoxFilterList)):
        for j in range(tempBoxFilterList[i].c,tempBoxFilterList[i].d+1):
            for k in range(tempBoxFilterList[i].a,tempBoxFilterList[i].b+1):
                #print(j,k)
                tempFilter[j][k] = tempFilter[j][k] + tempBoxFilterList[i].alpha

    return tempFilter
    



def calculateBoxFilters(filter,maxN):

    N=0
    maxN = maxN
    boundaries = []
    boxFilterList = []
    maxSize = filter.shape[0]
    aMax = maxSize
    bMax = maxSize
    cMax = maxSize
    dMax = maxSize




    boxFilterList = []
    minLoss = None
    minLossBox = None
    minLossBoxes = []
    flag = True
    while(N<maxN):
        
    
        #print("*****************************************************")
        #print("N:",N)
        for a in range(0,aMax):
            for b in range(0,bMax):
                for c in range(0,cMax):
                    for d in range(0,dMax):
                        #print("ABCD:",a,b,c,d)
                        tempBox = boxFilters(maxSize)
                        tempBox.setABCD(a,b,c,d)
                        #tempBox.printParams()
                        # print("tempBoxFilterList: Before")
                        # for i in tempBoxFilterList:
                        #  i.printParams() 
                        tempBoxFilterList = copy.deepcopy(boxFilterList)
                        tempBoxFilterList.append(tempBox)
                        #print("tempBoxFilterList: After")
                        # for i in tempBoxFilterList:
                        #     i.printParams() 

                        tempAlpha = calcAlpha(tempBoxFilterList,filter)
                        
                        if(np.all(tempAlpha)!=None):
                        #print("Alpha: ",tempAlpha)
                         for i,alpha in enumerate(tempAlpha):
                            
                            tempBoxFilterList[i].setAlpha(alpha)
                            #print(tempBoxFilterList[i].alpha)
                        #tempBox.setAlpha(tempAlpha[-1])
                        #print("tempBoxFilterList: After Alpha")
                        #  for i in tempBoxFilterList:
                        #   i.printParams() 
                         tempFilter = computeTempFilter(tempBoxFilterList)
                         loss = percentage_l2_error(tempFilter,filter)
                        #print("Loss:",loss)
                         if(minLoss==None):
                            minLoss = loss
                            minLossBox = tempBox
                            minLossBoxes = copy.deepcopy(tempBoxFilterList)
                         elif(loss<minLoss):
                            minLoss = loss
                            minLossBox = tempBox
                            minLossBoxes = copy.deepcopy(tempBoxFilterList)
                         if(np.isclose(minLoss,0)):
                            flag = False
                         if(not flag):
                            break
                    if(not flag):
                        break
                if(not flag):
                    break
            if(not flag):
                break
        if(not flag):
            break


        #print("-------------------------")
        print(N,"   ", minLoss)  
        if(minLossBox!=None):
         #print(minLoss,minLossBox.alpha,N)

         boxFilterList.append(minLossBox)
        N = N +1
    return minLossBoxes,minLoss                   

        

In [3]:
num_array = np.array([
    [1, 0, 1, 9, 6, 7, 9, 1, 7, 6, 0],
    [2, 7, 7, 5, 8, 2, 5, 8, 6, 1, 5],
    [7, 7, 4, 8, 9, 5, 1, 9, 9, 1, 3],
    [3, 6, 2, 9, 6, 7, 3, 5, 8, 2, 5],
    [3, 8, 1, 9, 9, 1, 3, 2, 7, 7, 5],
    [2, 5, 5, 8, 6, 1, 5, 9, 6, 7, 9],
    [2, 0, 9, 1, 7, 6, 0, 8, 6, 1, 5],
    [3, 8, 9, 5, 1, 9, 6, 2, 9, 6, 5],
    [5, 9, 6, 7, 3, 5, 8, 1, 9, 9, 7],
    [9, 9, 1, 3, 2, 5, 5, 8, 6, 1, 0]
])

filter = [[1,3,1,3,4,3,1],
          [7,4,7,4,5,4,7],
          [2,4,2,4,1,3,4],
          [7,4,5,4,7,4,5],
          [2,4,1,5,2,4,7]]

In [4]:
from scipy import signal
temp = np.array([[1,0,1,9,6,7,9],[2,7,7,5,8,2,5],[7,7,4,8,9,5,1],[3,6,2,9,6,7,3],[3,8,1,9,9,1,3],[2,5,5,8,6,1,5],[2,0,9,1,7,6,0]])
#filter = np.array([[0,0,1],[-1,0,-1],[0,0,1]])
output = signal.convolve2d(num_array, np.flip(filter), mode='valid')
print(output)

[[690 749 779 732 691]
 [702 758 760 771 732]
 [600 728 735 755 723]
 [647 662 742 696 714]
 [678 720 759 739 747]
 [685 659 764 657 678]]


In [17]:
filterSize = 5

#print(a.weight.data.shape)

filter = np.array([[1,1,1],[-2,-2,0],[-2,3,-3]])


boxFilterList,loss = calculateBoxFilters(filter,5)
print(loss)

0     163.2993161855452
1     55.63486402641868
2     32.71524934852403
3     14.586499149789454
2.474999889579944e-14


/home/csgrad/byalavar/miniconda3/envs/adaface/lib/python3.7/site-packages/ipykernel_launcher.py:121: RuntimeWarning: divide by zero encountered in double_scalars


In [9]:
for i in boxFilterList:
    i.printParams()

A: 1 B: 1 C: 2 D: 2 Alpha: 6.0
A: 0 B: 2 C: 1 D: 2 Alpha: -3.0000000000000004
A: 0 B: 2 C: 0 D: 1 Alpha: 1.0000000000000007
A: 2 B: 2 C: 1 D: 1 Alpha: 1.9999999999999998
A: 0 B: 0 C: 2 D: 2 Alpha: 1.0000000000000009


In [10]:
def calculate_summed_area_table(image):
    return np.cumsum(np.cumsum(image, axis=0), axis=1)



temp = np.array([[1,0,1,9,6,7,9],[2,7,7,5,8,2,5],[7,7,4,8,9,5,1],[3,6,2,9,6,7,3],[3,8,1,9,9,1,3],[2,5,5,8,6,1,5],[2,0,9,1,7,6,0]])

print(calculate_summed_area_table(temp))


[[  1   1   2  11  17  24  33]
 [  3  10  18  32  46  55  69]
 [ 10  24  36  58  81  95 110]
 [ 13  33  47  78 107 128 146]
 [ 16  44  59  99 137 159 180]
 [ 18  51  71 119 163 186 212]
 [ 20  53  82 131 182 211 237]]


In [32]:
kernel.shape

(5, 5, 5)

In [51]:
import numpy as np
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt

def gaussian_kernel_3d(sigma, noise_factor=0.05):
    # Define the size of the kernel
    size = 7
    
    # Generate the kernel using scipy's gaussian_filter function
    kernel = np.zeros((size, size))
    kernel[size // 2, size // 2] = 1
    kernel = gaussian_filter(kernel, sigma)

    # Normalize the kernel to ensure that the sum of all elements is equal to 1
    kernel /= np.sum(kernel)
    
    # Add random noise to the kernel
    noise = np.random.normal(0, noise_factor, kernel.shape)
    kernel += noise
    
    return kernel

# Example usage:
sigma = 2
kernel = gaussian_kernel_3d(sigma)
print("3D Gaussian Kernel with sigma =", sigma, ":\n", kernel)

3D Gaussian Kernel with sigma = 2 :
 [[ 0.04791525  0.07313959 -0.00385106 -0.06387007  0.03873513  0.05022171
  -0.0296984 ]
 [ 0.01538843  0.05408112  0.07680828  0.06167586  0.03120758  0.02599957
   0.04522766]
 [-0.01243138  0.07937869 -0.02333509  0.08134614  0.02823256 -0.05688994
   0.07171329]
 [-0.0041064  -0.03633534  0.11056809  0.04634317  0.04499057  0.02512878
  -0.06261741]
 [ 0.00522749  0.05238319  0.01224012  0.01154194 -0.07393869 -0.04257157
  -0.06435415]
 [ 0.02494503 -0.01903185  0.03148069  0.00431472 -0.02835172  0.03480126
   0.02708171]
 [ 0.05866982 -0.02804354  0.06969314  0.04876905  0.02860231 -0.00103015
   0.00020646]]


In [54]:
filterSize = 5
maxN = 32

#filter = np.random.random((filterSize,filterSize))
#conv = nn.Conv2d(1,1,filterSize)
#init.xavier_uniform_(conv.weight)
#filter = np.array(conv.weight.data[0][0])
filter = kernel
output = signal.convolve2d(temp, np.flip(filter), mode='valid')
boxFilterList,loss = calculateBoxFilters(filter,maxN)
approxFilter = computeTempFilter(boxFilterList)
output1 = signal.convolve2d(temp, np.flip(approxFilter), mode='valid')
print(loss)
print(percentage_l2_error(output,output1))

0     181.0416648536675
1     141.49688903335237
2     120.81600561862702
3     105.49005681197686
4     93.11942394848818
5     82.09174489582543
6     72.84019259403091
7     66.68906345481894
8     60.40718120787446
9     54.68278795226157
10     49.216923138970905
11     44.15479630249124
12     39.65627168354109
13     35.31705641418821
14     30.687744617608814
15     27.131277741185965
16     24.681291441387604
17     22.51320743462868
18     20.8376790800373
19     18.86446378280195
20     17.188182257533644
21     15.518370831288353
22     14.20915646224667
23     12.634368264690435
24     10.969116670628576
25     9.463611090590017
26     8.209343327338049
27     7.2432239789055455
28     5.538129704995162
29     4.620661899749791
30     3.974609754771203
31     3.3855085396573608
3.3855085396573608
0.17680825666317512


In [42]:
loss

3.14880279186479

In [2]:
#1D Box Filter

import numpy as np
import copy
import matplotlib.pyplot as plt

class boxFilters:
    def __init__(self,ogFilterSize):
        self.alpha = None
        self.a=None
        self.b=None
        #self.c=None
        #self.d=None
        self.ogFilterSize = ogFilterSize
    
    def setAlpha(self,alpha):
        self.alpha = alpha
    
    def setABCD(self,a,b):
        self.a = a
        self.b = b
        #self.c = c
        #self.d = d
    
    def printParams(self):
        print("A:",self.a,"B:",self.b, "Alpha:", self.alpha)


def overlapping_area(box1, box2):
    # Extract coordinates of the top-left and bottom-right corners of each box
    x1_tl,  x1_br  = box1.a, box1.b
    x2_tl,  x2_br = box2.a, box2.b 
    #print(x1_tl, y1_tl, x1_br, y1_br)
    #print(x2_tl, y2_tl, x2_br, y2_br)
    # Calculate coordinates of the intersection rectangle
    x_intersection_tl = max(x1_tl, x2_tl)
    #y_intersection_tl = max(y1_tl, y2_tl)
    x_intersection_br = min(x1_br, x2_br)
    #y_intersection_br = min(y1_br, y2_br)
    

    # Check if there is an actual intersection
    if x_intersection_tl <= x_intersection_br: #and y_intersection_tl <= y_intersection_br:
        # Calculate the width and height of the intersection rectangle
        intersection_width = x_intersection_br - x_intersection_tl + 1
        #intersection_height = y_intersection_br - y_intersection_tl + 1
        # Calculate the area of the intersection rectangle
       # print(intersection_width,intersection_height)
        intersection_area = intersection_width #* intersection_height
        return intersection_area
    else:
        # No intersection
        return 0

def sum_within_boundary(filter, x1, x2):
    # Ensure x1 <= x2 and y1 <= y2
    x1, x2 = min(x1, x2), max(x1, x2)
    #y1, y2 = min(y1, y2), max(y1, y2)

    # Ensure the boundary coordinates are within the matrix dimensions
    x1 = max(0, x1)
    x2 = min(len(filter) - 1, x2)
   # print("x1:",x1,"y1:",y1,"x2:",x2,"y2:",y2)
    # Calculate the sum within the boundary
    boundary_sum = 0

    for j in range(x1, x2 + 1):
            #print(i,j)
            boundary_sum += filter[j]
           # print(boundary_sum)
    return boundary_sum



def calcAlpha(boxFiltersList,filter):

    A = np.ones((len(boxFiltersList),len(boxFiltersList)))
    for i in range(0,len(boxFiltersList)):
        for j in range(0,len(boxFiltersList)):
           A[j][i]  =  overlapping_area(boxFiltersList[i],boxFiltersList[j])    
    
    det = np.linalg.det(A)
    #print("A:",A)
    AInv = None
    if np.isclose(det, 0):
        #print("Singular")
        pass
        
    else:
        # Calculate the inverse
        AInv = np.linalg.inv(A)
    
    B = np.ones((len(boxFiltersList)))

    for i in range(0,len(boxFiltersList)):
        B[i] = sum_within_boundary(filter,boxFiltersList[i].a,boxFiltersList[i].b)
    #print("B:",B)
    if(np.all(AInv==None)):
        return None
    else:
     return np.matmul(AInv,B)    
    

def l2_distance(arr1, arr2):
    # Flatten the arrays to treat them as vectors
    vec1 = arr1.flatten()
    vec2 = arr2.flatten()
    
    # Calculate the L2 distance
    distance = np.linalg.norm(vec1 - vec2)
    
    return distance

def percentage_l2_error(matrix1, matrix2):
    # Calculate the L2 norm of the difference between the matrices
    diff_norm = np.linalg.norm(matrix1 - matrix2)
    #print(diff_norm)
    # Calculate the L2 norm of one of the matrices
    norm_matrix1 = np.linalg.norm(matrix1)

    # Calculate the percentage L2 error
    percent_error = (diff_norm / norm_matrix1) * 100

    return percent_error



def computeTempFilter(tempBoxFilterList):
    tempFilter = np.zeros((tempBoxFilterList[0].ogFilterSize))
    for i in range(0,len(tempBoxFilterList)):
        #for j in range(tempBoxFilterList[i].c,tempBoxFilterList[i].d+1):
            for k in range(tempBoxFilterList[i].a,tempBoxFilterList[i].b+1):
                #print(j,k)
                tempFilter[k] = tempFilter[k] + tempBoxFilterList[i].alpha

    return tempFilter



def calculateBoxFilters(filter,maxN):

    N=0
    maxN = maxN
    boundaries = []
    boxFilterList = []
    maxSize = filter.shape[0]
    aMax = maxSize
    bMax = maxSize
    #cMax = maxSize
    #dMax = maxSize




    boxFilterList = []
    minLoss = None
    minLossBox = None
    minLossBoxes = []
    flag = True
    while(N<maxN):
        
    
        #print("*****************************************************")
        #print("N:",N)
        for a in range(0,aMax):
            for b in range(0,bMax):
                #for c in range(0,cMax):
                #    for d in range(0,dMax):
                        #print("ABCD:",a,b,c,d)
                        tempBox = boxFilters(maxSize)
                        tempBox.setABCD(a,b)
                        #tempBox.printParams()
                        # print("tempBoxFilterList: Before")
                        # for i in tempBoxFilterList:
                        #  i.printParams() 
                        tempBoxFilterList = copy.deepcopy(boxFilterList)
                        tempBoxFilterList.append(tempBox)
                        #print("tempBoxFilterList: After")
                        # for i in tempBoxFilterList:
                        #     i.printParams() 

                        tempAlpha = calcAlpha(tempBoxFilterList,filter)
                        
                        if(np.all(tempAlpha)!=None):
                        #print("Alpha: ",tempAlpha)
                         for i,alpha in enumerate(tempAlpha):
                            
                            tempBoxFilterList[i].setAlpha(alpha)
                            #print(tempBoxFilterList[i].alpha)
                        #tempBox.setAlpha(tempAlpha[-1])
                        #print("tempBoxFilterList: After Alpha")
                        #  for i in tempBoxFilterList:
                        #   i.printParams() 
                         tempFilter = computeTempFilter(tempBoxFilterList)
                         loss = (tempFilter,filter)
                         print("Loss:",loss)
                         if(minLoss==None):
                            minLoss = loss
                            minLossBox = tempBox
                            minLossBoxes = copy.deepcopy(tempBoxFilterList)
                         elif(loss<minLoss):
                            minLoss = loss
                            minLossBox = tempBox
                            minLossBoxes = copy.deepcopy(tempBoxFilterList)
                         if(np.isclose(minLoss,0)):
                            flag = False
                         if(not flag):
                            break
            if(not flag):
                break
        if(not flag):
            break


        #print("-------------------------")
        if(minLossBox!=None):
         #print(minLoss,minLossBox.alpha,N)

         boxFilterList.append(minLossBox)
        N = N +1
    return minLossBoxes,minLoss      

In [23]:
import torch.nn as nn



torch.Size([1, 1, 3, 3])


In [24]:
#filter = np.random.rand(25)



0     87.21060315365862
1     48.16434341098525
2     36.069607640072434
3     24.374600863882236
4     11.755535999923234
11.755535999923234


In [125]:
#3D Box Filters

# Helper Box Filters
import numpy as np
import copy
import matplotlib.pyplot as plt

class boxFilters:
    def __init__(self,ogFilterSize):
        self.alpha = None
        self.a=None
        self.b=None
        self.c=None
        self.d=None
        self.e=None
        self.f=None
        self.ogFilterSize = ogFilterSize
    
    def setAlpha(self,alpha):
        self.alpha = alpha
    
    def setABCD(self,a,b,c,d,e,f):
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        self.e = e
        self.f = f
    
    def printParams(self):
        print("A:",self.a,"B:",self.b,"C:",self.c,"D:",self.d,"E:",self.e,"F:",self.f,"Alpha:", self.alpha)


def overlapping_area(box1, box2):
    # Extract coordinates of the top-left and bottom-right corners of each box
    x1_tl, y1_tl, z1_tl, x1_br, y1_br, z1_br = box1.a, box1.c, box1.e, box1.b, box1.d, box1.f
    x2_tl, y2_tl, z2_tl, x2_br, y2_br, z2_br = box2.a, box2.c, box2.e, box2.b, box2.d, box2.f
    #print(x1_tl, y1_tl, x1_br, y1_br)
    #print(x2_tl, y2_tl, x2_br, y2_br)
    # Calculate coordinates of the intersection rectangle
    x_intersection_tl = max(x1_tl, x2_tl)
    y_intersection_tl = max(y1_tl, y2_tl)
    z_intersection_tl = max(z1_tl, z2_tl)
    x_intersection_br = min(x1_br, x2_br)
    y_intersection_br = min(y1_br, y2_br)
    z_intersection_br = min(z1_br, z2_br)
    

    # Check if there is an actual intersection
    if x_intersection_tl <= x_intersection_br and y_intersection_tl <= y_intersection_br and z_intersection_tl <= z_intersection_br:
        # Calculate the width and height of the intersection rectangle
        intersection_width = x_intersection_br - x_intersection_tl + 1
        intersection_height = y_intersection_br - y_intersection_tl + 1
        intersectrion_depth = z_intersection_br - z_intersection_tl + 1
        # Calculate the area of the intersection rectangle
       # print(intersection_width,intersection_height)
        intersection_area = intersection_width * intersection_height * intersectrion_depth
        return intersection_area
    else:
        # No intersection
        return 0

def sum_within_boundary(filter, x1, y1, z1, x2, y2, z2):
    # Ensure x1 <= x2 and y1 <= y2
    x1, x2 = min(x1, x2), max(x1, x2)
    y1, y2 = min(y1, y2), max(y1, y2)
    z1, z2 = min(z1, z2), max(z1, z2)

    # Ensure the boundary coordinates are within the matrix dimensions
    x1, y1, z1  = max(0, x1), max(0, y1), max(0, z1)
    x2, y2, z2 = min(len(filter) - 1, x2), min(len(filter[0]) - 1, y2), min(len(filter[0][0]) - 1, z2)
   # print("x1:",x1,"y1:",y1,"x2:",x2,"y2:",y2)
    # Calculate the sum within the boundary
    boundary_sum = 0
    for i in range(y1, y2 + 1):
        for j in range(x1, x2 + 1):
            for k in range(z1, z2 + 1):
            #print(i,j)
             boundary_sum += filter[i][j][k]
           # print(boundary_sum)
    return boundary_sum



def calcAlpha(boxFiltersList,filter):

    A = np.ones((len(boxFiltersList),len(boxFiltersList)))
    for i in range(0,len(boxFiltersList)):
        for j in range(0,len(boxFiltersList)):
           A[j][i]  =  overlapping_area(boxFiltersList[i],boxFiltersList[j])    
    
    det = np.linalg.det(A)
    #print("A:",A)
    AInv = None
    if np.isclose(det, 0):
        #print("Singular")
        pass
        
    else:
        # Calculate the inverse
        AInv = np.linalg.inv(A)
    
    B = np.ones((len(boxFiltersList)))

    for i in range(0,len(boxFiltersList)):
        B[i] = sum_within_boundary(filter,boxFiltersList[i].a,boxFiltersList[i].c,boxFiltersList[i].e,boxFiltersList[i].b,boxFiltersList[i].d,boxFiltersList[i].f)
    #print("B:",B)
    if(np.all(AInv==None)):
        return None
    else:
     return np.matmul(AInv,B)    
    

def l2_distance(arr1, arr2):
    # Flatten the arrays to treat them as vectors
    vec1 = arr1.flatten()
    vec2 = arr2.flatten()
    
    # Calculate the L2 distance
    distance = np.linalg.norm(vec1 - vec2)
    
    return distance

def percentage_l2_error(matrix1, matrix2):
    # Calculate the L2 norm of the difference between the matrices
    diff_norm = np.linalg.norm(matrix1 - matrix2)
    #print(diff_norm)
    # Calculate the L2 norm of one of the matrices
    norm_matrix1 = np.linalg.norm(matrix1)

    # Calculate the percentage L2 error
    percent_error = (diff_norm / norm_matrix1) * 100

    return percent_error



def computeTempFilter(tempBoxFilterList):
    tempFilter = np.zeros((tempBoxFilterList[0].ogFilterSize,tempBoxFilterList[0].ogFilterSize,tempBoxFilterList[0].ogFilterSize))
    for i in range(0,len(tempBoxFilterList)):
        for j in range(tempBoxFilterList[i].c,tempBoxFilterList[i].d+1):
            for k in range(tempBoxFilterList[i].a,tempBoxFilterList[i].b+1):
                for l in range(tempBoxFilterList[i].e,tempBoxFilterList[i].f+1):
                #print(j,k)
                    tempFilter[j][k][l] = tempFilter[j][k][l] + tempBoxFilterList[i].alpha

    return tempFilter
    



def calculateBoxFilters(filter,maxN):

    N=0
    maxN = maxN
    boundaries = []
    boxFilterList = []
    maxSize = filter.shape[0]
    aMax = maxSize
    bMax = maxSize
    cMax = maxSize
    dMax = maxSize
    eMax = maxSize 
    fMax = maxSize




    boxFilterList = []
    minLoss = None
    minLossBox = None
    minLossBoxes = []
    flag = True
    while(N<maxN):
        
    
        #print("*****************************************************")
        #print("N:",N)
        for a in range(0,aMax):
            for b in range(0,bMax):
                for c in range(0,cMax):
                    for d in range(0,dMax):
                      for e in range(0,eMax):
                       for f in range(0,fMax):   
                        #print("ABCD:",a,b,c,d)
                        tempBox = boxFilters(maxSize)
                        tempBox.setABCD(a,b,c,d,e,f)
                        #tempBox.printParams()
                        # print("tempBoxFilterList: Before")
                        # for i in tempBoxFilterList:
                        #  i.printParams() 
                        tempBoxFilterList = copy.deepcopy(boxFilterList)
                        tempBoxFilterList.append(tempBox)
                        #print("tempBoxFilterList: After")
                        # for i in tempBoxFilterList:
                        #     i.printParams() 

                        tempAlpha = calcAlpha(tempBoxFilterList,filter)
                        
                        if(np.all(tempAlpha)!=None):
                        #print("Alpha: ",tempAlpha)
                         for i,alpha in enumerate(tempAlpha):
                            
                            tempBoxFilterList[i].setAlpha(alpha)
                            #print(tempBoxFilterList[i].alpha)
                        #tempBox.setAlpha(tempAlpha[-1])
                        #print("tempBoxFilterList: After Alpha")
                        #  for i in tempBoxFilterList:
                        #   i.printParams() 
                         tempFilter = computeTempFilter(tempBoxFilterList)
                         loss = percentage_l2_error(tempFilter,filter)
                         #print("Loss:",loss)
                         if(minLoss==None):
                            minLoss = loss
                            minLossBox = tempBox
                            minLossBoxes = copy.deepcopy(tempBoxFilterList)
                         elif(loss<minLoss):
                            minLoss = loss
                            minLossBox = tempBox
                            minLossBoxes = copy.deepcopy(tempBoxFilterList)
                         if(np.isclose(minLoss,0)):
                            flag = False
                         if(not flag):
                            break
                       if(not flag):
                        break
                      if(not flag):
                        break  
                    if(not flag):
                        break
                if(not flag):
                    break
            if(not flag):
                break
        if(not flag):
            break

        print(N, minLoss)
        #print("-------------------------")
        if(minLossBox!=None):
         #print(minLoss,minLossBox.alpha,N)

         boxFilterList.append(minLossBox)
        N = N +1
    return minLossBoxes,minLoss                   

        

In [126]:

filterSize = 5
maxN = 125
conv = nn.Conv3d(1,1,filterSize)
init.xavier_uniform_(conv.weight)
#print(conv.weight.data.shape)
#filter = np.array(conv.weight.data[0][0])
#filter = np.random.uniform(-5,5,filterSize)
filter = kernel
boxFilterList,loss = calculateBoxFilters(filter,maxN)

0 270.47329317285596
1 204.27744799021013
2 169.2018074101956
3 144.78783686570358
4 127.26180897418855
5 114.77504627530656
6 105.05521411522989
7 97.14478258316456
8 91.47598835966382
9 84.174156328346
10 79.35097634641392
11 75.06551627913969
12 71.01372724462482
13 67.46352629338952
14 63.73559108773953
15 60.80042381222685
16 58.1197138361373
17 55.603032434095425
18 52.89362220456806
19 50.53666373875407
20 48.36616636093649
21 46.32800201796336
22 44.3806734943649
23 42.633090007252896
24 40.94335713824906
25 38.78315899186336
26 37.34714254417047
27 35.71095919305517
28 34.32129741294057
29 32.64611897877961
30 31.18215666938562
31 29.816464054569618
32 28.526502558321088
33 27.195564507277375
34 25.83876572469816
35 24.450262733170927
36 23.15380736478927
37 22.162624278141948
38 20.983294272455613
39 19.987499888370802
40 19.003083224835677
41 18.14244056254217
42 17.307667399023714
43 16.581599758019458
44 15.939924453389457
45 15.280950419131312
46 14.597672400308213
47 13.

In [185]:
filter = np.random.rand(5,5,5)

In [186]:
filter

array([[[0.22562617, 0.20612485, 0.09397523, 0.9371723 , 0.19257183],
        [0.90215191, 0.7654766 , 0.08134264, 0.68518063, 0.03161184],
        [0.16456519, 0.25583328, 0.90495855, 0.25948673, 0.17062913],
        [0.95514863, 0.04462158, 0.36988211, 0.63356039, 0.27132475],
        [0.25084936, 0.32695998, 0.74600693, 0.03557023, 0.03505262]],

       [[0.79324526, 0.27270328, 0.82985252, 0.21621186, 0.74996947],
        [0.28046505, 0.86535748, 0.26725503, 0.96193962, 0.72273838],
        [0.03532112, 0.87400072, 0.35610974, 0.52500547, 0.3443158 ],
        [0.59299081, 0.01319214, 0.69113468, 0.47244196, 0.07087485],
        [0.06685432, 0.339805  , 0.86609922, 0.04459604, 0.45481749]],

       [[0.6424782 , 0.8149455 , 0.02188115, 0.2125553 , 0.13809142],
        [0.74663596, 0.34641223, 0.83470841, 0.84707848, 0.21699626],
        [0.27863588, 0.36416365, 0.39025722, 0.609461  , 0.26847536],
        [0.4924467 , 0.4618505 , 0.55917064, 0.20543452, 0.19009206],
        [0.80026

In [190]:
boxes,loss = calculateBoxFilters(filter,90)
print(loss)

0 67.08270337595563
1 63.274046060651656
2 60.105879775684144
3 57.41397855022241
4 55.2509839151321
5 53.345639637895204
6 51.50732256139422
7 49.670243311329244
8 47.80146114920482
9 45.755652529284184
10 43.78210682496204
11 41.88955938690256
12 39.35788599780762
13 37.193043420787696
14 35.00339890491932
15 32.65099893762816
16 30.81398163437996
17 29.220190142383267
18 27.535236856823058
19 26.341271916002277
20 25.1053572259762
21 24.048785200563234
22 23.112554600468318
23 22.202970281453222
24 21.342944680903784
25 20.473392939759805
26 19.70129400562132
27 19.012580012038715
28 18.191748852513822
29 17.445651290032316
30 16.72925592260332
31 16.06913183504184
32 15.350214185487651
33 14.719781777372651
34 13.975356320770402
35 13.300947573041416
36 13.300947573041416
37 13.300947573041416
38 13.300947573041416
39 13.300947573041416
40 13.300947573041416
41 13.300947573041416
42 13.300947573041416
43 13.300947573041416
44 13.300947573041416
45 13.300947573041416
46 13.300947573

KeyboardInterrupt: 

In [183]:
approxFilter = computeTempFilter(boxes)
print(approxFilter,percentage_l2_error(filter,approxFilter))

1.3348042682986183
[[[0.25913615 0.25913615 0.57880416 0.16427921 0.75859067]
  [0.25913615 0.25913615 0.57880416 0.57880416 0.57880416]
  [0.01752773 0.57880416 0.03028036 0.57880416 0.57880416]
  [0.57880416 0.57880416 0.03028036 0.57880416 0.57880416]
  [0.57880416 0.14535125 0.57880416 0.57880416 0.57880416]]

 [[0.57880416 0.57880416 0.57880416 0.16427921 0.75859067]
  [0.57880416 0.99087282 0.21006973 0.57880416 0.57880416]
  [0.01752773 0.21006973 0.43314903 0.80188346 0.57880416]
  [1.03730308 0.66856865 0.15985447 0.02642121 0.3055096 ]
  [0.1665136  0.57880416 0.3055096  0.3055096  0.3055096 ]]

 [[0.57880416 0.57880416 0.75961181 0.93939833 0.93939833]
  [0.57880416 0.21006973 0.39087738 0.75961181 0.4257697 ]
  [0.57880416 0.66594768 0.61395669 0.98269111 0.75961181]
  [0.57880416 0.21006973 0.15985447 0.02642121 0.3055096 ]
  [0.1665136  0.13940795 0.82102716 0.3055096  0.3055096 ]]

 [[0.57880416 0.83705919 1.01786684 0.93939833 0.93939833]
  [0.17623986 0.83705919 0.4135

In [86]:
conv3 = torch.nn.Conv3d(1,1,2,padding=0,bias=False)
conv3.weight.data = torch.tensor(torch.ones(1,1,2,2,2))

a = torch.rand(1,1,3,3,3)
b = conv3(a)

print(a)
print(b)

tensor([[[[[0.8226, 0.5114, 0.3643],
           [0.9287, 0.7610, 0.3765],
           [0.1129, 0.3964, 0.5899]],

          [[0.2202, 0.9278, 0.1764],
           [0.4328, 0.4336, 0.6001],
           [0.4016, 0.4502, 0.9085]],

          [[0.9935, 0.4534, 0.0029],
           [0.7808, 0.8997, 0.2696],
           [0.2277, 0.2736, 0.1857]]]]])
tensor([[[[[5.0382, 4.1512],
           [3.9171, 4.5163]],

          [[5.1419, 3.7637],
           [3.9000, 4.0211]]]]], grad_fn=<ConvolutionBackward0>)


/home/csgrad/byalavar/miniconda3/envs/adaface/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [110]:
import numpy as np

def calculate_3d_integral_image(image):
    x_len = len(image)
    y_len = len(image[0])
    z_len = len(image[0][0])
    print(x_len,y_len,z_len)
    integral_image = [[[0 for _ in range(z_len)] for _ in range(y_len)] for _ in range(x_len)]

    for x in range(x_len):
        for y in range(y_len):
            for z in range(z_len):
                sum = image[x][y][z]

                if x > 0:
                    sum += integral_image[x-1][y][z]
                if y > 0:
                    sum += integral_image[x][y-1][z]
                if z > 0:
                    sum += integral_image[x][y][z-1]
                if x > 0 and y > 0 and z > 0:
                    sum -= integral_image[x-1][y-1][z-1]

                integral_image[x][y][z] = sum

    return integral_image


sat = calculate_3d_integral_image(np.ones((3,3,3)))

print(sat)

3 3 3
[[[1.0, 2.0, 3.0], [2.0, 5.0, 9.0], [3.0, 9.0, 19.0]], [[2.0, 5.0, 9.0], [5.0, 15.0, 32.0], [9.0, 32.0, 79.0]], [[3.0, 9.0, 19.0], [9.0, 32.0, 79.0], [19.0, 79.0, 223.0]]]


In [115]:
x = np.ones((3,3,3)).cumsum(0).cumsum(1).cumsum(2)

print(x)

[[[ 1.  2.  3.]
  [ 2.  4.  6.]
  [ 3.  6.  9.]]

 [[ 2.  4.  6.]
  [ 4.  8. 12.]
  [ 6. 12. 18.]]

 [[ 3.  6.  9.]
  [ 6. 12. 18.]
  [ 9. 18. 27.]]]


In [112]:
np.ones((3,3,3))

array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]])

In [120]:
def calculate_3d_integral_image(image):
    x_len = len(image)
    y_len = len(image[0])
    z_len = len(image[0][0])

    integral_image = [[[0 for _ in range(z_len)] for _ in range(y_len)] for _ in range(x_len)]

    for x in range(x_len):
        for y in range(y_len):
            for z in range(z_len):
                sum = image[x][y][z]

                if x > 0:
                    sum += integral_image[x-1][y][z]
                if y > 0:
                    sum += integral_image[x][y-1][z]
                if z > 0:
                    sum += integral_image[x][y][z-1]
                if x > 0 and y > 0:
                    sum -= integral_image[x-1][y-1][z]
                if x > 0 and z > 0:
                    sum -= integral_image[x-1][y][z-1]
                if y > 0 and z > 0:
                    sum -= integral_image[x][y-1][z-1]
                if x > 0 and y > 0 and z > 0:
                    sum += integral_image[x-1][y-1][z-1]

                integral_image[x][y][z] = sum

    return integral_image

# Create a 3x3x3 array filled with ones
x = [[[1 for _ in range(3)] for _ in range(3)] for _ in range(3)]
integral_image = calculate_3d_integral_image(x)

integral_image = np.array(integral_image)

print(np.all(integral_image == ))

TypeError: 'tuple' object cannot be interpreted as an integer

In [140]:
a[1][1][1]

0.893099727437831

In [135]:
a = np.random.random((3,3,3))

In [136]:
np.array(calculate_3d_integral_image(a))

array([[[ 0.77376205,  1.66948762,  2.01586054],
        [ 1.39059627,  2.74423624,  3.8895595 ],
        [ 1.72572448,  3.37095541,  4.9033893 ]],

       [[ 1.08603565,  2.10915163,  2.47990213],
        [ 2.63275956,  5.00688967,  6.40759727],
        [ 3.09515277,  6.01691845,  8.27067709]],

       [[ 1.63593448,  3.50936361,  4.33214316],
        [ 4.1486665 ,  8.03465817, 10.85255325],
        [ 4.61601253,  9.82202545, 13.84537397]]])

In [137]:
a.cumsum(0).cumsum(1).cumsum(2)

array([[[ 0.77376205,  1.66948762,  2.01586054],
        [ 1.39059627,  2.74423624,  3.8895595 ],
        [ 1.72572448,  3.37095541,  4.9033893 ]],

       [[ 1.08603565,  2.10915163,  2.47990213],
        [ 2.63275956,  5.00688967,  6.40759727],
        [ 3.09515277,  6.01691845,  8.27067709]],

       [[ 1.63593448,  3.50936361,  4.33214316],
        [ 4.1486665 ,  8.03465817, 10.85255325],
        [ 4.61601253,  9.82202545, 13.84537397]]])

In [138]:
out = np.array(calculate_3d_integral_image(a))

In [134]:
np.all(out==a.cumsum(0).cumsum(1).cumsum(2))

False

In [145]:
a = np.ones((3,3,3))
print(a)

[[[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]]


In [73]:
def sum_within_boundary(integral_image, corner1, corner2):
    x1, y1, z1 = corner1
    x2, y2, z2 = corner2

    total = integral_image[x2][y2][z2]

    if x1 > 0:
        total -= integral_image[x1-1][y2][z2]
    if y1 > 0:
        total -= integral_image[x2][y1-1][z2]
    if z1 > 0:
        total -= integral_image[x2][y2][z1-1]
    if x1 > 0 and y1 > 0:
        total += integral_image[x1-1][y1-1][z2]
    if x1 > 0 and z1 > 0:
        total += integral_image[x1-1][y2][z1-1]
    if y1 > 0 and z1 > 0:
        total += integral_image[x2][y1-1][z1-1]
    if x1 > 0 and y1 > 0 and z1 > 0:
        total -= integral_image[x1-1][y1-1][z1-1]

    return total

def check_dimensions(sat, corner1, corner2):
    x_len = len(sat)
    y_len = len(sat[0])
    z_len = len(sat[0][0])
    print(x_len,y_len,z_len)
    x1, y1, z1 = corner1
    x2, y2, z2 = corner2

    if not (0 <= x1 < x_len and 0 <= x2 < x_len):
        return False
    if not (0 <= y1 < y_len and 0 <= y2 < y_len):
        return False
    if not (0 <= z1 < z_len and 0 <= z2 < z_len):
        return False

    return True



In [146]:
out = calculate_3d_integral_image(a)

In [154]:
a

array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]])

In [148]:
out = np.array(out)
print(out)

[[[ 1.  2.  3.]
  [ 2.  4.  6.]
  [ 3.  6.  9.]]

 [[ 2.  4.  6.]
  [ 4.  8. 12.]
  [ 6. 12. 18.]]

 [[ 3.  6.  9.]
  [ 6. 12. 18.]
  [ 9. 18. 27.]]]


In [153]:
if check_dimensions(out, (1,1,1), (2,2,2)):
    print(sum_within_boundary(out, (1,1,1), (2,2,2)))
else:
    print("Coordinates are out of range")

3 3 3
8.0


In [75]:
sat

[[[tensor([[0.8350, 0.5906, 0.1387],
           [0.2284, 0.6240, 0.6502],
           [0.3236, 0.0805, 0.9270]]),
   tensor([[1.0435, 1.0450, 0.5815],
           [0.4497, 1.5152, 0.9564],
           [0.6222, 1.0607, 1.0949]]),
   tensor([[1.9781, 1.2663, 1.0074],
           [1.0812, 2.1144, 1.7920],
           [1.4459, 1.7816, 1.2045]])]]]

In [54]:
len(filter)

25

In [55]:
tempFilter = computeTempFilter(boxFilterList)

In [56]:
percentage_l2_error(filter,tempFilter)

1.8910732587444037

In [40]:
for i in boxFilterList:
    i.printParams()

A: 7 B: 21 Alpha: 60.64935064935048
A: 14 B: 18 Alpha: -46.03896103896096
A: 8 B: 10 Alpha: -46.10389610389596
A: 15 B: 20 Alpha: 32.76623376623374
A: 8 B: 15 Alpha: 34.77272727272728
A: 13 B: 14 Alpha: -27.740259740259717
A: 6 B: 11 Alpha: -17.519480519480396
A: 14 B: 19 Alpha: -17.766233766233825
A: 3 B: 7 Alpha: 24.941558441558414
A: 3 B: 9 Alpha: -7.999999999999993
A: 3 B: 17 Alpha: -9.46103896103898
A: 13 B: 21 Alpha: -4.649350649350481
A: 5 B: 13 Alpha: 2.662337662337606
A: 3 B: 8 Alpha: -2.999999999999975
A: 0 B: 1 Alpha: 1.5


In [29]:
tempFilter

array([  1.,   2.,  -1.,   4.,   5.,   7., -10.,  50.,  14.,  17.])

In [7]:
model = resnet20()

loaded_data = torch.load('/home/csgrad/byalavar/FHE/ImprovedLT/HEAAN/app/homomorphic_dft/resnet20-12fca82f.th')

state_dict = loaded_data['state_dict']
if 'module' in list(state_dict.keys())[0]:
    state_dict = {k[7:]: v for k, v in state_dict.items()}

model.load_state_dict(state_dict)

<All keys matched successfully>

In [8]:
model

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

In [37]:
for i,j in model.named_parameters():
    print(i)

conv1.weight
bn1.weight
bn1.bias
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.2.conv1.weight
layer1.2.bn1.weight
layer1.2.bn1.bias
layer1.2.conv2.weight
layer1.2.bn2.weight
layer1.2.bn2.bias
layer2.0.conv1.weight
layer2.0.bn1.weight
layer2.0.bn1.bias
layer2.0.conv2.weight
layer2.0.bn2.weight
layer2.0.bn2.bias
layer2.1.conv1.weight
layer2.1.bn1.weight
layer2.1.bn1.bias
layer2.1.conv2.weight
layer2.1.bn2.weight
layer2.1.bn2.bias
layer2.2.conv1.weight
layer2.2.bn1.weight
layer2.2.bn1.bias
layer2.2.conv2.weight
layer2.2.bn2.weight
layer2.2.bn2.bias
layer3.0.conv1.weight
layer3.0.bn1.weight
layer3.0.bn1.bias
layer3.0.conv2.weight
layer3.0.bn2.weight
layer3.0.bn2.bias
layer3.1.conv1.weight
layer3.1.bn1.weight
layer3.1.bn1.bias
layer3.1.conv2.weight
layer3.1.bn2.weight
layer3.1.bn2.bias
layer3.

In [18]:
#Input image


transform = transforms.Compose([
    #transforms.Resize((224,224)),
    transforms.ToTensor(),
    
    transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])
])

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False, num_workers=2)


input = next(iter(testloader))
input = input[0]
print(input.shape)
inputImage = input
print(inputImage.shape)


Files already downloaded and verified
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])


In [80]:


def getIntermediateOutput(model, input, layer_name):
    
    def hook(module, input, output_from_hook):
        nonlocal output
        output = output_from_hook

    # Assume `model` is your PyTorch model
    output = None


    layer = getattr(model, layer_name)
    print(layer)
# Register the hook on the 'conv1' layer
    handle = layer.register_forward_hook(hook)

# Run your model
    print(input.shape)
    model(input)

# Now `output` contains the output of the 'conv1' layer
    #print(output.shape)

# Don't forget to remove the hook when you're done
    handle.remove()


    list_array = output[0].tolist()

# Convert the nested list to a JSON string
    json_str = json.dumps(list_array)

# Write the JSON string to a file
    with open('output'+str(layer_name)+'.json', 'w') as f:
        f.write(json_str)



In [81]:
getIntermediateOutput(model,inputImage,'conv1')

Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
torch.Size([1, 3, 32, 32])


In [49]:
def calculateBoxes(model,layerName):
    approxFiltersList = []
    boxes = []
    avgLoss = 0
    count = 0
    for name,params in model.named_parameters():
     if(layerName == str(name) and "weight" in str(name)):

      print(name)  
      approxFilters = np.zeros(params.shape)
      print(params.shape)
      data = params.data
      for i in range(0,data.shape[0]):
          for j in range(0,data.shape[1]): 
              boxFilterList,loss = calculateBoxFilters(data[i][j].numpy(),5)
              boxes.append(boxFilterList)
              print(i,loss)
              avgLoss = avgLoss + loss
              count = count + 1
              approxFilters[i][j] = computeTempFilter(boxFilterList)
      approxFiltersList.append(approxFilters)
     else:
        break 
     

    print("Avg Loss:",avgLoss/count)
    return boxes,approxFiltersList

def combineBoxes(boxes,filterId):
    alpha = [boxes[i].alpha for i in range(0,len(boxes))]
    a = [boxes[i].a for i in range(0,len(boxes))]
    b = [boxes[i].b for i in range(0,len(boxes))]
    c = [boxes[i].c for i in range(0,len(boxes))]
    d = [boxes[i].d for i in range(0,len(boxes))]
    return {
            'alpha': alpha,
            'a': a,
            'b': b,
            'c': c,
            'd': d,
            'ogFilterSize': boxes[0].ogFilterSize,
            'filterId': filterId
        }



def saveFilters(boxes,layerName):
   box_filters_dicts = [combineBoxes(bf,i) for i,bf in enumerate(boxes)]
   with open(str(layerName)+'BoxFilters'+'.json', 'w') as f:
    json.dump(box_filters_dicts, f)


def updateModel(model,approxFiltersList,layerName,count):
    for name,params in model.named_parameters():
     if(layerName == str(name) and "weight" in str(name)):
      print(name)
    
      params.data = torch.tensor(approxFiltersList[count],dtype=torch.float32)

    return model

In [48]:
boxes,approxFiltersList = calculateBoxes(model,"conv1.weight")
saveFilters(boxes,"conv1.weight")

conv1.weight
torch.Size([16, 3, 3, 3])
0 0.24862375398432798
0 0.1557454432545499
0 0.22847514483480855
1 0.08145631606391394
1 0.09750105796518906
1 0.15523216255744213
2 0.17600194457403023
2 0.06061386439070259
2 0.027944698198377062
3 0.06559950631376948
3 0.10756782715852824
3 0.07348432794265856
4 0.018300600924455377
4 0.020270806770108927
4 0.07306367806780141
5 0.0028278242800427625
5 0.0041105831427531644
5 0.006973680159042001
6 0.00235678779215978
6 0.001949890358286074
6 0.002218207385238251
7 0.22863143746666273
7 0.060097475310513446
7 0.15254108900727992
8 0.03047110059978478
8 0.04305545887079639
8 0.01012613081039765
9 0.14498951788381953
9 0.07428156984365739
9 0.015436445437252411
10 0.05786094067445617
10 0.12409591744174572
10 0.06518190120230212
11 0.12271480703857432
11 0.12867753405850185
11 0.12069126713869192
12 0.02217655944837466
12 0.09240493711836402
12 0.09189114788573967
13 0.08398428521872882
13 0.15466068428406846
13 0.09815484372607616
14 0.001052303

In [52]:
model = updateModel(model,approxFiltersList,"conv1.weight",0)

conv1.weight


In [53]:
getIntermediateOutput(model,inputImage,'conv1')

Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
torch.Size([1, 3, 32, 32])


In [82]:
output

tensor([[[[ 1.7325e-01, -1.4064e+00, -1.2660e+00,  ..., -1.6951e+00,
           -1.7315e+00, -3.2145e+00],
          [ 1.1140e+00, -6.4385e-01, -7.0539e-01,  ..., -9.0847e-01,
           -1.1075e+00, -2.1459e+00],
          [ 9.3316e-01, -7.3117e-01, -7.6620e-01,  ..., -7.1212e-01,
           -8.4256e-01, -2.3040e+00],
          ...,
          [-1.0957e+00,  3.8449e-01, -1.3411e-01,  ...,  1.1548e+00,
            8.8465e-02,  1.4916e+00],
          [-1.7750e+00,  5.6020e-01, -1.8068e-01,  ..., -3.5378e-01,
            1.2851e+00,  5.3959e-01],
          [-2.6705e+00, -7.5113e-01, -1.1901e+00,  ..., -2.0702e+00,
           -1.1226e+00, -9.9885e-01]],

         [[ 2.6363e-02, -5.0788e-01, -6.5977e-01,  ..., -1.2288e+00,
           -1.2744e+00,  5.7595e-01],
          [ 9.6250e-01,  4.8289e-01,  3.8867e-01,  ..., -4.5533e-01,
           -4.9840e-01,  6.7859e-01],
          [ 8.7263e-01,  3.5216e-01,  2.9549e-01,  ..., -2.9318e-01,
           -5.6844e-01,  6.4111e-01],
          ...,
     

In [93]:
import torch
import json

def getBatchNormWeights(model, layer_name, filename):
    # Get the parameters of the model
    model.eval()
    weights = {'weights': j.tolist() for i, j in model.named_parameters() if layer_name+'.weight'== i}
    bias = {'bias': j.tolist() for i, j in model.named_parameters() if layer_name+'.bias' == i}
    
    layer = getattr(model, layer_name)
    runningMean = {'runningMean':layer.running_mean.tolist()}
    runningVar = {'runningVar':layer.running_var.tolist()}

    paramsList = [weights,bias,runningMean,runningVar]

    # Dump the weights to a JSON file
    with open(filename, 'w') as f:
        json.dump(paramsList, f)
    
  


getBatchNormWeights(model, 'bn1', 'bn1.json')

In [59]:
for i,j in model.named_parameters():
    print(i)
    if(i=='bn1.weight'):
        print(j)

conv1.weight
bn1.weight
Parameter containing:
tensor([6.1040e-01, 6.6255e-01, 6.3536e-01, 7.2446e-01, 9.3609e-01, 1.5853e-01,
        1.1061e-03, 6.2115e-01, 7.5787e-01, 6.5871e-01, 6.8427e-01, 1.1494e+00,
        5.9217e-01, 3.7244e-01, 9.2601e-05, 6.6238e-01], requires_grad=True)
bn1.bias
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.2.conv1.weight
layer1.2.bn1.weight
layer1.2.bn1.bias
layer1.2.conv2.weight
layer1.2.bn2.weight
layer1.2.bn2.bias
layer2.0.conv1.weight
layer2.0.bn1.weight
layer2.0.bn1.bias
layer2.0.conv2.weight
layer2.0.bn2.weight
layer2.0.bn2.bias
layer2.1.conv1.weight
layer2.1.bn1.weight
layer2.1.bn1.bias
layer2.1.conv2.weight
layer2.1.bn2.weight
layer2.1.bn2.bias
layer2.2.conv1.weight
layer2.2.bn1.weight
layer2.2.bn1.bias
layer2.2.conv2.weight
layer2.2.bn2.weight
layer2

In [85]:
model.eval()
getIntermediateOutput(model,inputImage,'bn1')

BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
torch.Size([1, 3, 32, 32])


In [79]:
output.shape

torch.Size([1, 16, 32, 32])

In [68]:
model.bn1.weight.shape

torch.Size([16])

In [83]:
def manual_batch_norm_2d_eval(x, gamma, beta, running_mean, running_var, eps=1e-5):
    x_normalized = (x - running_mean.view(1, -1, 1, 1)) / torch.sqrt(running_var.view(1, -1, 1, 1) + eps)
    gamma = gamma.view(1, -1, 1, 1)
    beta = beta.view(1, -1, 1, 1)
    return gamma * x_normalized + beta

# Example usage:


y = manual_batch_norm_2d_eval(output, model.bn1.weight, model.bn1.bias, model.bn1.running_mean, model.bn1.running_var)

torch.Size([16])

In [76]:
model.bn1.running_mean

tensor([-0.0183, -0.0501, -0.4938, -0.2989, -0.1088, -0.0676, -0.0080,  0.7018,
        -0.4636, -0.0198,  0.0608,  0.2817,  0.1115, -0.2711, -0.0022, -0.1840])

In [84]:
y

tensor([[[[ 1.2023e+00,  4.3823e-01,  5.0610e-01,  ...,  2.9856e-01,
            2.8097e-01, -4.3637e-01],
          [ 1.6573e+00,  8.0705e-01,  7.7728e-01,  ...,  6.7906e-01,
            5.8276e-01,  8.0518e-02],
          [ 1.5698e+00,  7.6481e-01,  7.4787e-01,  ...,  7.7403e-01,
            7.1093e-01,  4.0478e-03],
          ...,
          [ 5.8850e-01,  1.3045e+00,  1.0536e+00,  ...,  1.6770e+00,
            1.1613e+00,  1.8400e+00],
          [ 2.5993e-01,  1.3894e+00,  1.0311e+00,  ...,  9.4736e-01,
            1.7401e+00,  1.3795e+00],
          [-1.7325e-01,  7.5516e-01,  5.4285e-01,  ...,  1.1713e-01,
            5.7548e-01,  6.3534e-01]],

         [[ 9.7215e-01,  7.6035e-01,  7.0013e-01,  ...,  4.7454e-01,
            4.5645e-01,  1.1900e+00],
          [ 1.3433e+00,  1.1531e+00,  1.1158e+00,  ...,  7.8118e-01,
            7.6411e-01,  1.2307e+00],
          [ 1.3077e+00,  1.1013e+00,  1.0788e+00,  ...,  8.4547e-01,
            7.3634e-01,  1.2159e+00],
          ...,
     

In [96]:
y.shape

torch.Size([1, 16, 32, 32])

In [117]:
#ReLU

def getReLUScalingFactors(input, layer):
    abs_input = torch.abs(input)
    max_values, _ = torch.max(abs_input.view(input.shape[1], -1), dim=1)
    max_values_list = max_values.tolist()

    with open('reluScaling_'+str(layer)+'.json', 'w') as f:
        json.dump(max_values_list, f)

In [119]:
y

tensor([[[[ 1.2023e+00,  4.3823e-01,  5.0610e-01,  ...,  2.9856e-01,
            2.8097e-01, -4.3637e-01],
          [ 1.6573e+00,  8.0705e-01,  7.7728e-01,  ...,  6.7906e-01,
            5.8276e-01,  8.0518e-02],
          [ 1.5698e+00,  7.6481e-01,  7.4787e-01,  ...,  7.7403e-01,
            7.1093e-01,  4.0478e-03],
          ...,
          [ 5.8850e-01,  1.3045e+00,  1.0536e+00,  ...,  1.6770e+00,
            1.1613e+00,  1.8400e+00],
          [ 2.5993e-01,  1.3894e+00,  1.0311e+00,  ...,  9.4736e-01,
            1.7401e+00,  1.3795e+00],
          [-1.7325e-01,  7.5516e-01,  5.4285e-01,  ...,  1.1713e-01,
            5.7548e-01,  6.3534e-01]],

         [[ 9.7215e-01,  7.6035e-01,  7.0013e-01,  ...,  4.7454e-01,
            4.5645e-01,  1.1900e+00],
          [ 1.3433e+00,  1.1531e+00,  1.1158e+00,  ...,  7.8118e-01,
            7.6411e-01,  1.2307e+00],
          [ 1.3077e+00,  1.1013e+00,  1.0788e+00,  ...,  8.4547e-01,
            7.3634e-01,  1.2159e+00],
          ...,
     

In [118]:
getReLUScalingFactors(y,0)

In [115]:
def getReLUOutput(input,layer):
    x_relu = torch.where(input > 0, input, torch.zeros_like(input))
    print(x_relu.shape)
    x_relu = x_relu[0].tolist()

    with open('reluOut'+str(layer)+'.json', 'w') as f:
        json.dump(x_relu, f)

In [116]:
getReLUOutput(y,0)

torch.Size([1, 16, 32, 32])
